# Sudoku Solver (v3)

Close to final form.

Objectives:

1. Assess effectiveness and performance against different test cases
 1. Back tracking
 2. Back tracking + constraint propogation
 3. Solution X or Dancing Links?
2. Analyse our performance
3. Draw some pretty graphs
4. Assess performance against other implementations


## Supporting Packages

In [1]:
import sudoku as su
import puzzlegrid as pg
import pandas as pd
from IPython.display import HTML, display, clear_output

pd.set_option('precision', 3)

## Help functions

Print puzzles.

In [2]:
def print_puzzle(puzzle):
    display(HTML(puzzle.as_html()))

def print_2_puzzles(puz1, puz2):
    display(HTML('<table><tr><td>' + puz1.as_html() + '</td><td>' + puz2.as_html() + '</td></tr></table>'))

def update_progress(label, current, total, time_so_far, test_case):
    clear_output(wait=True)
    display(HTML(f'<progress style="width: 100%" max={total} value={current}>{current} out of {total}</progress>'))
    if test_case:
        display(HTML(f"<p>Working on {label}: <i>{test_case['label']}</i>, total time so far {time_so_far:.2f} seconds</p>"))
    else:
        display(HTML(f"<p>Completed in {time_so_far:.2f} seconds</p>"))
    return

## Example class usage

Create a puzzle in `SudokuPuzzle` then use separate class `SudokuSolver`. Separation FTW.


In [3]:
p = su.SudokuPuzzle()
p.init_puzzle(su.SAMPLE_PUZZLES[0]['puzzle'])
solver = su.SudokuSolver(p)
solver.solve()
print_2_puzzles(solver.original, p)

8,9,,4,,,,5,6
1,4,,3,5,,,9,
,,,,,,8,,
9,,,,,,2,,
,8,,9,6,5,,4,
,,1,,,,,,5
,,8,,,,,,
,3,,,2,1,,7,8
4,2,,,,6,,1,3
8,9,3,4,7,2,1,5,6
1,4,6,3,5,8,7,9,2


# Sudoku Solution Strategies

## Backtracking and Constraint Propogation



In [4]:
include_levels = ['Kids', 'Easy', 'Moderate'] #, 'Hard', 'Diabolical', 'Pathalogical']
test_cases = [x for x in su.SAMPLE_PUZZLES if x['level'] in include_levels]
p = su.SudokuPuzzle()

In [5]:
pt = pg.PuzzleTester(puzzle_class=su.SudokuPuzzle, test_samples=3)
pt.add_testcases(test_cases)

for s in su.SOLVERS:
    solver = su.SudokuSolver(p, method=s)
    pt.run_tests(solver, s, callback=update_progress)

In [6]:
df = pd.DataFrame(pt.get_test_results())
df

,label,level,starting_clues,constraintpropogation (SudokuSolver),backtracking (SudokuSolver)
0,SMH 1,Kids,31,0.002,0.003
1,SMH 2,Easy,24,0.003,0.187
2,KTH 1,Easy,30,0.002,0.012
3,Rico Alan Heart,Easy,22,0.028,0.076
4,SMH 3,Moderate,26,0.024,0.088


Add more test cases from (insert source)

In [7]:
pt.add_testcases(pg.from_file("data/hardest.txt", level="Hard"))
pt.add_testcases(pg.from_file("data/top95.txt", level="Diabolical"))
for s in su.SOLVERS:
    solver = su.SudokuSolver(p, method=s)
    pt.run_tests(solver, s, callback=update_progress)

In [9]:
df = pd.DataFrame(pt.get_test_results())
df

,label,level,starting_clues,constraintpropogation (SudokuSolver),backtracking (SudokuSolver)
0,SMH 1,Kids,31,0.001,0.003
1,SMH 2,Easy,24,0.002,0.178
2,KTH 1,Easy,30,0.003,0.012
3,Rico Alan Heart,Easy,22,0.032,0.078
4,SMH 3,Moderate,26,0.032,0.084
...,...,...,...,...,...
106,data/top95.txt:91,Diabolicalii,23,0.003,0.212
107,data/top95.txt:92,Diabolicalii,23,0.008,0.151
108,data/top95.txt:93,Diabolicalii,22,0.046,4.272
109,data/top95.txt:94,Diabolicalii,23,0.019,5.519


In [11]:
df.to_pickle('sudoku_test_results.pkl')


# Appendix

## Sources

Part of this exercise was to learn Python and Jupyter skills while also solving a problem that I found interesting. So I've largely avoided reading other people's solutions to solving Sudoku. However from time to time I've gotten stuck or just been curious about something and found the below sources useful.

* The "Top 95" and "Hardest" puzzle examples in the data directory come from [Solving Every Sudoku Puzzle (by Peter Norvig)](https://norvig.com/sudoku.html)
* [Sudoku solving algorithms](https://en.wikipedia.org/wiki/Sudoku_solving_algorithms) -- links to some sample puzzles (on Flickr of all places). Found via the [Wikipedia article on Sudoku solving algorithms](https://en.wikipedia.org/wiki/Sudoku_solving_algorithms).
* [AI Sudoku](http://www.aisudoku.com/index_en.html) -- collection of really hard puzzles.
* The [sudoku.py](sudoku.py) class has URLs to where I found some of the sample puzzles. I've attempted to use labels for them that credit the source, although it's not always clear where the original puzzle came from.
* Also used examples from [Simple sudoku solver using constraint propagation](https://gpicavet.github.io/jekyll/update/2017/12/16/sudoku-solver.html) (Grégory Picavet's Blog).


## Table formatting

Snippet below inserts some CSS to make the table look more like a Sudoku puzzle grid.


In [ ]:
display(HTML('''
<style type="text/css">
.sudoku table {
    border: 3px solid red;
}

.sudoku td {
    width: 40px;
    height: 40px;
    border: 1px solid #F00;
    text-align: center;
}

.sudoku td:nth-of-type(3n) {    
    border-right: 3px solid red;
}

.sudoku tr:nth-of-type(3n) td {    
    border-bottom: 3px solid red;
}

.sudoku-solved table {
    border: 3px solid green;
}

.sudoku-solved td {
    border: 1pm solid green !important;
}

</style>
'''))

Scratch.txt

#10: Rico Alan Border #1
Greg: Solved in 3 ms, 59 tests, 0 backtracks
Me: Solved in 3 ms

#13: Rico Alan #3
Greg: Invalid board!
Me: Solved in 1.771 s

#11: Rico Alan #4
Greg: Solved in 30 ms, 5396 tests, 2371 backtracks
Me: Solved in 26 ms

#12: Qassim Hamza
Greg: Solved in 15 ms
Me: Solved in 59 ms

#14: World's Hardest Sudoku 2012
Greg: Solved in 21 ms
Me: Solved in 30 ms

#15: AI escargot
Greg: Solved in 1 ms, 187 tests, 12 backtracks
Me: Solved in 4 ms